# Bike rental data Cleaning 

### Library and Data Imports

In [107]:
import pandas as pd
import datetime
import numpy as np
import glob

In [108]:
#creating list of filenames using glob library
files = glob.glob("./data/JC-******-citibike-tripdata.csv")
#iterating through the list of filenames and appending data frames to the list
df_list=[]
for filename in files:
    data = pd.read_csv(filename)
    df_list.append(data)
bikes_df = pd.concat(df_list)
#importing the weather file
weather_df = pd.read_csv('./data/newark_airport_2016.csv')


## Citi Bike Data Overview

In [109]:
bikes_df.head(3)

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,361,2016-02-01 00:31:18,2016-02-01 00:37:19,3202,Newport PATH,40.727224,-74.033759,3203,Hamilton Park,40.727596,-74.044247,24393,Subscriber,1975.0,1
1,297,2016-02-01 01:55:05,2016-02-01 02:00:02,3195,Sip Ave,40.730743,-74.063784,3194,McGinley Square,40.725340,-74.067622,24394,Subscriber,1985.0,2
2,1155,2016-02-01 02:40:05,2016-02-01 02:59:20,3183,Exchange Place,40.716247,-74.033459,3210,Pershing Field,40.742677,-74.051789,24676,Subscriber,1976.0,1


In [110]:
bikes_df.tail(3)

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
19485,2048,2016-05-31 23:25:28,2016-05-31 23:59:36,3199,Newport Pkwy,40.728745,-74.032108,3199,Newport Pkwy,40.728745,-74.032108,24573,Subscriber,1975.0,1
19486,455,2016-05-31 23:31:57,2016-05-31 23:39:32,3220,5 Corners Library,40.734961,-74.059503,3215,Central Ave,40.746730,-74.049251,24561,Subscriber,1964.0,1
19487,239,2016-05-31 23:47:38,2016-05-31 23:51:37,3185,City Hall,40.717732,-74.043845,3211,Newark Ave,40.721525,-74.046305,24685,Subscriber,1993.0,1


* There are 19487 rows in the data set
* Trip Duration is in seconds
* Gender can be 0=unknown, 1=male, and 2=female
* User Type has values Customer=24 hour pass or 3 day user and Subscriber=Annual Member

* max Trip Duration looks like outlier
* min Birth Year is very small

In [111]:
# checking for missing values
bikes_df.isna().sum()

Trip Duration                  0
Start Time                     0
Stop Time                      0
Start Station ID               0
Start Station Name             0
Start Station Latitude         0
Start Station Longitude        0
End Station ID                 0
End Station Name               0
End Station Latitude           0
End Station Longitude          0
Bike ID                        0
User Type                    380
Birth Year                 18999
Gender                         0
dtype: int64

* User type - mising data
* Birth year - missing data
* Gender - data are not missing, but we know from data dictionary, that O represent "unknown"

In [112]:
# checking for duplicates
bikes_df.duplicated().sum()

0

We have no duplicates, which is good

In [113]:
bikes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 247584 entries, 0 to 19487
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Trip Duration            247584 non-null  int64  
 1   Start Time               247584 non-null  object 
 2   Stop Time                247584 non-null  object 
 3   Start Station ID         247584 non-null  int64  
 4   Start Station Name       247584 non-null  object 
 5   Start Station Latitude   247584 non-null  float64
 6   Start Station Longitude  247584 non-null  float64
 7   End Station ID           247584 non-null  int64  
 8   End Station Name         247584 non-null  object 
 9   End Station Latitude     247584 non-null  float64
 10  End Station Longitude    247584 non-null  float64
 11  Bike ID                  247584 non-null  int64  
 12  User Type                247204 non-null  object 
 13  Birth Year               228585 non-null  float64
 14  Gender    

Almost all datatypes are good, except Start and Stop time

## Citi Bike Data Transformation

#### Columns to investigate:

* Start Time/Stop Time: wrong data types.
* Trip Duration: looks like there are outliers.
* Birth Year: missing data and possible outliers.
* Gender: investigate unknowns.
* User type: missing data.


#### Changing columns names for comfortable work

In [114]:
bikes_df.columns = bikes_df.columns.str.lower().str.replace(' ', '_')
bikes_df.columns

Index(['trip_duration', 'start_time', 'stop_time', 'start_station_id',
       'start_station_name', 'start_station_latitude',
       'start_station_longitude', 'end_station_id', 'end_station_name',
       'end_station_latitude', 'end_station_longitude', 'bike_id', 'user_type',
       'birth_year', 'gender'],
      dtype='object')

Looks good, let's start transformation

#### Start Time/ Stop Time

In [115]:
# casting objects as timestamps
bikes_df.start_time = pd.to_datetime(bikes_df.start_time,infer_datetime_format=True)
bikes_df.stop_time = pd.to_datetime(bikes_df.stop_time,infer_datetime_format=True)
bikes_df.dtypes

/var/folders/pn/9t8hjz551fxgxlw6qx40xhl40000gn/T/ipykernel_2519/4149122457.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  bikes_df.start_time = pd.to_datetime(bikes_df.start_time,infer_datetime_format=True)
/var/folders/pn/9t8hjz551fxgxlw6qx40xhl40000gn/T/ipykernel_2519/4149122457.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  bikes_df.stop_time = pd.to_datetime(bikes_df.stop_time,infer_datetime_format=True)


trip_duration                       int64
start_time                 datetime64[ns]
stop_time                  datetime64[ns]
start_station_id                    int64
start_station_name                 object
start_station_latitude            float64
start_station_longitude           float64
end_station_id                      int64
end_station_name                   object
end_station_latitude              float64
end_station_longitude             float64
bike_id                             int64
user_type                          object
birth_year                        float64
gender                              int64
dtype: object

#### Trip Duration

In [116]:
# creating an easier to interpret trip time column in minutes and hours
bikes_df['trip_minutes'] = round(bikes_df['trip_duration'] / 60, 2)
bikes_df['trip_hours'] = round(bikes_df['trip_duration']/(60**2),2)
bikes_df[['trip_duration','trip_minutes','trip_hours']].describe()

,trip_duration,trip_minutes,trip_hours
count,2.475840e+05,247584.000000,247584.000000
mean,8.856305e+02,14.760503,0.246006
std,3.593798e+04,598.966283,9.982777
min,6.100000e+01,1.020000,0.020000
25%,2.480000e+02,4.130000,0.070000
50%,3.900000e+02,6.500000,0.110000
75%,6.660000e+02,11.100000,0.180000
max,1.632981e+07,272163.470000,4536.060000


* minimum trip length = 61 seconds
* maximum trip = 4,536 hours =189 days

According to the data dictionary, trips under 60 seconds were already assumed to be false starts/stops and were pruned.

For the max, Citi Bike policies state that the maximum length of a trip is 24 hours (or 86400 seconds).

We won't delete the longer trips, since it is possible these correspond to bike docking issues or users breaking the rules (which analysts might want to investigate). However, we will add a flag so that the analytics team can easily filter these out if desired.

In [117]:
# create a column to flag rides that do not comply with system protocols
bikes_df['valid_duration'] = bikes_df.trip_duration.apply(lambda x: 0 if x > 86400 else 1).astype(bool)

#### Birth Year

In [118]:
#calculating age from birth years
bikes_df['age'] = pd.Timestamp.now().year - bikes_df.birth_year
bikes_df.age.sort_values(ascending=False).head(10)

4417     124.0
22758     90.0
6163      87.0
2929      87.0
32771     87.0
16672     87.0
23319     84.0
2201      84.0
29718     84.0
7314      83.0
Name: age, dtype: float64

smb is 123 years old in 2023 year, so I guess we can delete this row from the observation.

Now let's have a look at missing data

In [119]:
birth_year_missing_data = bikes_df[bikes_df.birth_year.isnull()]

In [120]:
birth_year_missing_data.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,trip_minutes,trip_hours,valid_duration,age
125,3665,2016-02-01 09:56:46,2016-02-01 10:57:52,3212,Christ Hospital,40.734786,-74.050444,3185,City Hall,40.717732,-74.043845,24531,Customer,NaN,0,61.08,1.02,True,NaN
148,1081,2016-02-01 11:43:51,2016-02-01 12:01:53,3183,Exchange Place,40.716247,-74.033459,3192,Liberty Light Rail,40.711242,-74.055701,24624,Customer,NaN,0,18.02,0.30,True,NaN
154,1100,2016-02-01 12:04:35,2016-02-01 12:22:56,3192,Liberty Light Rail,40.711242,-74.055701,3192,Liberty Light Rail,40.711242,-74.055701,24624,Customer,NaN,0,18.33,0.31,True,NaN
163,1579,2016-02-01 12:26:09,2016-02-01 12:52:29,3192,Liberty Light Rail,40.711242,-74.055701,3183,Exchange Place,40.716247,-74.033459,24624,Customer,NaN,0,26.32,0.44,True,NaN
168,1619,2016-02-01 12:53:44,2016-02-01 13:20:44,3186,Grove St PATH,40.719586,-74.043117,3203,Hamilton Park,40.727596,-74.044247,24510,Customer,NaN,0,26.98,0.45,True,NaN


As I can see from these 5 rows: In every row, where birth_year is missing, gender is missing also. Additionally, user_type for this rows is always 'Customer'. Let's check if this pattern will cover the rest of the dataset.

In [121]:
print('Length of the Dataframe with missing birth years:')
display(len(birth_year_missing_data))
print('User type value counts:')
display(birth_year_missing_data.user_type.value_counts(dropna=False))
print('Gender value counts:')
display(birth_year_missing_data.gender.value_counts())

Length of the Dataframe with missing birth years:


18999

User type value counts:


user_type
Customer      15470
Subscriber     3529
Name: count, dtype: int64

Gender value counts:


gender
0    18999
Name: count, dtype: int64

* As we can see: all genders are missing.That means that if there is no birth_year ther is also no gender data.
* the customer user_type seems to be missing a lot of birth_year data
Let's check the percentages for user_type:

In [122]:
round(birth_year_missing_data['user_type'].value_counts() / bikes_df['user_type'].value_counts(),4)*100

user_type
Customer      99.67
Subscriber     1.52
Name: count, dtype: float64

* 99% of customers are missing birth_year
* 1.5% of subscribers are missing birth_year
* all records with missing birth_year also have unknown gender, which likely also corresponds to the customer user_type
We won't delete records that are missing birth_year, since that would remove all customers. We'll have to keep this missing data in mind when developing the database.

#### user_type

In [123]:
round(bikes_df.user_type.value_counts(dropna=False) / len(bikes_df.user_type),4)*100

user_type
Subscriber    93.58
Customer       6.27
NaN            0.15
Name: count, dtype: float64

Less then 1% is missing, so actually we can delete these rows, but let's have a deeper look, maybe birth_year is also missing and we will be able to consider missing user_types as 'Customers'.

In [124]:
missing_user_type_data = bikes_df[bikes_df.user_type.isnull()]

In [125]:
missing_user_type_data.birth_year.isnull().sum()

0

There is no missing birth_year, so missing user_types are not 'Customers'. We can delete this rows, but I will better mark them as unknown:

In [126]:
bikes_df.fillna({'user_type':'Unknown'},inplace=True)

In [127]:
bikes_df.reset_index(drop=True,inplace=True)
bikes_df['id'] = bikes_df.index
bikes_df.tail(3)

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,trip_minutes,trip_hours,valid_duration,age,id
247581,2048,2016-05-31 23:25:28,2016-05-31 23:59:36,3199,Newport Pkwy,40.728745,-74.032108,3199,Newport Pkwy,40.728745,-74.032108,24573,Subscriber,1975.0,1,34.13,0.57,True,49.0,247581
247582,455,2016-05-31 23:31:57,2016-05-31 23:39:32,3220,5 Corners Library,40.734961,-74.059503,3215,Central Ave,40.746730,-74.049251,24561,Subscriber,1964.0,1,7.58,0.13,True,60.0,247582
247583,239,2016-05-31 23:47:38,2016-05-31 23:51:37,3185,City Hall,40.717732,-74.043845,3211,Newark Ave,40.721525,-74.046305,24685,Subscriber,1993.0,1,3.98,0.07,True,31.0,247583


### Weather Data Overview

In [128]:
weather_df.head(3)

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
0,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-01,12.75,NaN,0.0,0.0,0.0,41,43,34,NaN,270,280.0,25.9,35.1
1,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-02,9.40,NaN,0.0,0.0,0.0,36,42,30,NaN,260,260.0,21.0,25.1
2,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-03,10.29,NaN,0.0,0.0,0.0,37,47,28,NaN,270,250.0,23.9,30.0


In [129]:
weather_df.tail(3)

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
363,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-12-29,8.05,NaN,0.36,0.0,0.0,38,45,31,NaN,170,150.0,18.1,25.1
364,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-12-30,14.99,NaN,0.00,0.0,0.0,37,42,32,NaN,270,270.0,25.9,33.1
365,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-12-31,12.30,NaN,0.00,0.0,0.0,35,44,29,NaN,200,220.0,21.9,28.0


In [130]:
weather_df.describe()

,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
count,366.000000,0.0,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,0.0,366.000000,364.000000,366.000000,364.000000
mean,9.429973,NaN,0.104945,0.098087,0.342623,57.196721,65.991803,48.459016,NaN,217.841530,228.269231,20.484426,26.801648
std,3.748174,NaN,0.307496,1.276498,2.078510,17.466981,18.606301,17.135790,NaN,102.548282,97.415777,6.848390,8.882610
min,2.460000,NaN,0.000000,0.000000,0.000000,8.000000,18.000000,0.000000,NaN,10.000000,10.000000,6.900000,10.100000
25%,6.765000,NaN,0.000000,0.000000,0.000000,43.000000,51.250000,35.000000,NaN,150.000000,150.000000,15.000000,19.900000
50%,8.720000,NaN,0.000000,0.000000,0.000000,56.000000,66.000000,47.000000,NaN,240.000000,260.000000,19.900000,25.100000
75%,11.410000,NaN,0.030000,0.000000,0.000000,74.000000,83.000000,64.000000,NaN,300.000000,300.000000,23.900000,31.100000
max,22.820000,NaN,2.790000,24.000000,20.100000,89.000000,99.000000,80.000000,NaN,360.000000,360.000000,48.100000,66.000000


### Cleaning and Transforming Weather Data

PGTM (peak gust time) and TSUN (total minutes of sun) are entirely missing and will be dropped. WDFx and WSFx refer to the fastest x-minute wind, and are difficult to connect directly to our data (since we don't know when that speed occurred.) We'll therefore stick with AWND (average wind speed) for our wind-speed variable.

Also, since all our data is from the same station, we'll drop those details for now.

In [131]:
weather_df.drop(['STATION','NAME','PGTM','TSUN', 'WDF2', 'WDF5', 'WSF2', 'WSF5'],axis=1, inplace=True)

Let's rename the columns to make the data clearer

In [132]:
weather_df.columns = ['rec_date','avg_wind','prcp','snow_amt','snow_depth','tavg','tmax','tmin']

Lastly, let's add some new columns to assist analysts

In [133]:
# create convenience binary columns for weather events
weather_df['rain'] = weather_df['prcp'].apply(lambda x: 1 if x > 0 else 0).astype(bool)
weather_df['snow'] = weather_df['snow_amt'].apply(lambda x: 1 if x > 0 else 0).astype(bool)

In [134]:
weather_df.head()

,rec_date,avg_wind,prcp,snow_amt,snow_depth,tavg,tmax,tmin,rain,snow
0,2016-01-01,12.75,0.0,0.0,0.0,41,43,34,False,False
1,2016-01-02,9.40,0.0,0.0,0.0,36,42,30,False,False
2,2016-01-03,10.29,0.0,0.0,0.0,37,47,28,False,False
3,2016-01-04,17.22,0.0,0.0,0.0,32,35,14,False,False
4,2016-01-05,9.84,0.0,0.0,0.0,19,31,10,False,False


Let's have a look at data types

In [135]:
weather_df.dtypes

rec_date       object
avg_wind      float64
prcp          float64
snow_amt      float64
snow_depth    float64
tavg            int64
tmax            int64
tmin            int64
rain             bool
snow             bool
dtype: object

Everything looks ok, except 'rec_date'

In [136]:
weather_df.rec_date = pd.to_datetime(weather_df.rec_date,infer_datetime_format=True)

/var/folders/pn/9t8hjz551fxgxlw6qx40xhl40000gn/T/ipykernel_2519/1210891316.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  weather_df.rec_date = pd.to_datetime(weather_df.rec_date,infer_datetime_format=True)


# Adding other dimensional tables

### Date Dimensional Table

In [137]:
# our data starts on 2016-01-01
start = datetime.datetime(2016,1,1)
# generate the remaining days of the year -- leap year!
date_col = [start + datetime.timedelta(x) for x in range(366)]
# create a dataframe
df_date = pd.DataFrame(date_col, dtype='datetime64[ns]')
df_date.columns = ['full_date']
df_date.head()

,full_date
0,2016-01-01
1,2016-01-02
2,2016-01-03
3,2016-01-04
4,2016-01-05


In [138]:
# generate month and day names
df_date['month'] = df_date['full_date'].apply(lambda x: x.month)
df_date['day'] = df_date['full_date'].apply(lambda x: x.day)
df_date['month_name'] = df_date['full_date'].apply(lambda x: x.strftime('%B'))
df_date['day_name'] = df_date['full_date'].apply(lambda x: x.strftime('%A'))
df_date['weekend'] = df_date['day_name'].apply(lambda x: 1 if (x == 'Saturday' or x=='Sunday') else 0).astype(bool)
df_date.head()

,full_date,month,day,month_name,day_name,weekend
0,2016-01-01,1,1,January,Friday,False
1,2016-01-02,1,2,January,Saturday,True
2,2016-01-03,1,3,January,Sunday,True
3,2016-01-04,1,4,January,Monday,False
4,2016-01-05,1,5,January,Tuesday,False


We'll want to use this date dimension table in joins, but joining on date-time columns is risky. Let's add a date key to each of our tables.

In [139]:
df_date['date_key'] = df_date['full_date'].apply(lambda x: int(x.strftime('%Y%m%d').strip('-')))
weather_df['date_key'] = weather_df['rec_date'].apply(lambda x: int(x.strftime('%Y%m%d').strip('-')))
bikes_df['date_key'] = bikes_df['start_time'].apply(lambda x: int(x.strftime('%Y%m%d').strip('-')))

# take a look at the date table
df_date.head()

,full_date,month,day,month_name,day_name,weekend,date_key
0,2016-01-01,1,1,January,Friday,False,20160101
1,2016-01-02,1,2,January,Saturday,True,20160102
2,2016-01-03,1,3,January,Sunday,True,20160103
3,2016-01-04,1,4,January,Monday,False,20160104
4,2016-01-05,1,5,January,Tuesday,False,20160105


In [140]:
weather_df.head()

,rec_date,avg_wind,prcp,snow_amt,snow_depth,tavg,tmax,tmin,rain,snow,date_key
0,2016-01-01,12.75,0.0,0.0,0.0,41,43,34,False,False,20160101
1,2016-01-02,9.40,0.0,0.0,0.0,36,42,30,False,False,20160102
2,2016-01-03,10.29,0.0,0.0,0.0,37,47,28,False,False,20160103
3,2016-01-04,17.22,0.0,0.0,0.0,32,35,14,False,False,20160104
4,2016-01-05,9.84,0.0,0.0,0.0,19,31,10,False,False,20160105


### Demographics Table

Let's create a table for storing demographic information.

In [141]:
# create a demographics table with all the unique demographics in the original dataset
trip_demo = bikes_df[['user_type','birth_year','gender','age']].drop_duplicates(subset=['user_type','birth_year','gender']).reset_index(drop=True)
trip_demo['trip_demo'] = trip_demo.index

# add the trip_demo key to a rides table
rides = bikes_df.merge(trip_demo, on=['user_type','birth_year','gender','age']).sort_values(by='id').reset_index(drop=True)

# drop the demographic info from rides
rides = rides.drop(['user_type','birth_year','gender','age'],axis=1)

rides.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,trip_minutes,trip_hours,valid_duration,id,date_key,trip_demo
0,361,2016-02-01 00:31:18,2016-02-01 00:37:19,3202,Newport PATH,40.727224,-74.033759,3203,Hamilton Park,40.727596,-74.044247,24393,6.02,0.10,True,0,20160201,0
1,297,2016-02-01 01:55:05,2016-02-01 02:00:02,3195,Sip Ave,40.730743,-74.063784,3194,McGinley Square,40.725340,-74.067622,24394,4.95,0.08,True,1,20160201,1
2,1155,2016-02-01 02:40:05,2016-02-01 02:59:20,3183,Exchange Place,40.716247,-74.033459,3210,Pershing Field,40.742677,-74.051789,24676,19.25,0.32,True,2,20160201,2
3,1769,2016-02-01 05:11:28,2016-02-01 05:40:58,3214,Essex Light Rail,40.712774,-74.036486,3203,Hamilton Park,40.727596,-74.044247,24700,29.48,0.49,True,3,20160201,3
4,935,2016-02-01 05:48:24,2016-02-01 06:03:59,3203,Hamilton Park,40.727596,-74.044247,3214,Essex Light Rail,40.712774,-74.036486,24639,15.58,0.26,True,4,20160201,3


Because we've created a new key, let's double check that merging with the demographics table reproduces the original dataframe.

In [142]:

# first merge the two tables and
# - sort by ID to match bikes_df's sorting
# - reset the index to match bikes_df's index
# - drop the trip_demo join key
ridesmerge = rides.merge(trip_demo,on='trip_demo').sort_values(by='id').reset_index(drop=True).drop('trip_demo',axis=1)

# count the number of positions where ridesmerge is different from bikes_df
# note that NaN != NaN, so we also need to make sure at least one isn't NaN
(bikes_df.ne(ridesmerge) & (bikes_df.notna() | ridesmerge.notna())).sum()

age                        0
bike_id                    0
birth_year                 0
date_key                   0
end_station_id             0
end_station_latitude       0
end_station_longitude      0
end_station_name           0
gender                     0
id                         0
start_station_id           0
start_station_latitude     0
start_station_longitude    0
start_station_name         0
start_time                 0
stop_time                  0
trip_duration              0
trip_hours                 0
trip_minutes               0
user_type                  0
valid_duration             0
dtype: int64

In [143]:
# change the name of the trip_demo id column for the database
trip_demo.rename(columns={'trip_demo':'id'}, inplace=True)
trip_demo.head()

,user_type,birth_year,gender,age,id
0,Subscriber,1975.0,1,49.0,0
1,Subscriber,1985.0,2,39.0,1
2,Subscriber,1976.0,1,48.0,2
3,Subscriber,1974.0,2,50.0,3
4,Subscriber,1984.0,2,40.0,4


### Station Table

Let's create a table of stations. These already have IDs, so we just need to create a new table with station id, name, latitude, and longitude. The easiest way to make sure we have all the necessary records is simply to make a table of all possible station records and drop duplicates.

In [144]:
# create tables of start stations and end stations
starts = rides[['start_station_id','start_station_name','start_station_latitude','start_station_longitude']]
ends = rides[['end_station_id','end_station_name','end_station_latitude','end_station_longitude']]

# rename so both start and end dataframes have the same column names
starts.columns = ['id','station_name','latitude','longitude']
ends.columns = ['id','station_name','latitude','longitude']

# concatenate and drop duplicates
stations = pd.concat([starts,ends]).drop_duplicates()

# drop station names and locations from rides
rides = rides.drop(['start_station_name','start_station_latitude','start_station_longitude','end_station_name','end_station_latitude','end_station_longitude'],axis=1)

rides.head()

,trip_duration,start_time,stop_time,start_station_id,end_station_id,bike_id,trip_minutes,trip_hours,valid_duration,id,date_key,trip_demo
0,361,2016-02-01 00:31:18,2016-02-01 00:37:19,3202,3203,24393,6.02,0.10,True,0,20160201,0
1,297,2016-02-01 01:55:05,2016-02-01 02:00:02,3195,3194,24394,4.95,0.08,True,1,20160201,1
2,1155,2016-02-01 02:40:05,2016-02-01 02:59:20,3183,3210,24676,19.25,0.32,True,2,20160201,2
3,1769,2016-02-01 05:11:28,2016-02-01 05:40:58,3214,3203,24700,29.48,0.49,True,3,20160201,3
4,935,2016-02-01 05:48:24,2016-02-01 06:03:59,3203,3214,24639,15.58,0.26,True,4,20160201,3


In [145]:
stations.head()

,id,station_name,latitude,longitude
0,3202,Newport PATH,40.727224,-74.033759
1,3195,Sip Ave,40.730743,-74.063784
2,3183,Exchange Place,40.716247,-74.033459
3,3214,Essex Light Rail,40.712774,-74.036486
4,3203,Hamilton Park,40.727596,-74.044247


In [146]:
stations.to_csv("./csv/stations.csv", index=False)

In [147]:

new_order = ['id', 'date_key','trip_duration', 'trip_minutes', 'trip_hours', 'start_time', 'stop_time', 'start_station_id', 'end_station_id', 'bike_id', 'valid_duration', 'trip_demo']
rides = rides[new_order]

rides.to_csv("./csv/rides.csv", index=False)

In [148]:
new_order = ['id', 'user_type', 'gender', 'birth_year', 'age']
trip_demo = trip_demo[new_order]
trip_demo.to_csv("./csv/trip_demo.csv", index=False)

In [149]:

weather_df.insert(0, 'id', range(len(weather_df)))

new_order = ['id', 'rec_date', 'avg_wind', 'prcp', 'snow_amt', 'snow_depth', 'tavg', 'tmax', 'tmin', 'date_key','rain', 'snow']
weather_df = weather_df[new_order]

weather_df.to_csv("./csv/weather.csv", index=False)

In [150]:
new_order = ['date_key', 'full_date', 'month', 'day', 'month_name', 'day_name','weekend']
df_date = df_date[new_order]


df_date.to_csv("./csv/date_dim.csv", index=False)